# League of Legends v4.20 Deep Reinforcement Learning Agent

## Introduction

This Google Colab Notebook is created as a self-contained example for running League of Legends v4.20 DRL training with [pylol](https://github.com/MiscellaneousStuff/pylol) library.
To use it, simply execute each cell below one by one.

# Installation

## GameServer (from source)

#### Install .NET

In [ ]:
!wget https://packages.microsoft.com/config/ubuntu/18.04/packages-microsoft-prod.deb -O packages-microsoft-prod.deb
!sudo dpkg -i packages-microsoft-prod.deb

#### Install the Runtime

In [ ]:
!sudo apt-get update; \
  sudo apt-get install -y apt-transport-https && \
  sudo apt-get update && \
  sudo apt-get install -y dotnet-sdk-3.1

#### Clone GameServer

In [ ]:
!rm -rf LeagueSandbox-RL-Learning
!git clone https://github.com/MiscellaneousStuff/LeagueSandbox-RL-Learning
!cd LeagueSandbox-RL-Learning && git checkout master && git branch && git submodule init && git submodule update

### Build GameServer

In [ ]:
!cd LeagueSandbox-RL-Learning  && dotnet build .

#### Run GameServer to Generate Configs

In [ ]:
!cd /content/LeagueSandbox-RL-Learning/GameServerConsole/bin/Debug/netcoreapp3.0/ && \
/content/LeagueSandbox-RL-Learning/GameServerConsole/bin/Debug/netcoreapp3.0/GameServerConsole --redis_port 6379

## PyLoL (from source)

In [ ]:
!git clone -v https://github.com/MiscellaneousStuff/pylol.git
!pip3 install --upgrade pylol/

## Test PyLoL

### Install Redis Client (Python) and Redis Server (Linux)

In [ ]:
!pip3 install redis
!sudo apt-get install redis-server

#### Write Config Dirs (just GameServer for now)

In [ ]:
# Only need to write config for GameServer here as we don't use the League Client within Colab environment
!touch config_dirs.txt
!printf "[dirs]\ngameserver = /content/LeagueSandbox-RL-Learning/GameServerConsole/bin/Debug/netcoreapp3.0/\nlolclient = " > config_dirs.txt

### Run Test

#### Test PyLoL on Local IP

In [ ]:
# Get local IP
host = !hostname -i
host = host[0]

# Run Client
!python3 -m pylol.bin.client --max_steps 100 --host $host --agent scripted --config_path "config_dirs.txt"

#### Test PyLoL with a Custom Agent

In [ ]:
# Imports
from pylol.env import lol_env
from pylol.env import run_loop
from pylol.agents import base_agent
from pylol.lib import actions

# Settings
feature_map_size = 16000
feature_move_range = 8
player_list = "Ezreal.BLUE,Ezreal.PURPLE" # Comma-separated list of `Player.Team`
map = "Old Summoners Rift" # ["New Summoners Rift", "Howling Abyss"]
max_steps = 100 # 100 steps / 7.5 steps (a second) = 13.33 seconds
max_episodes = 0 # When set to 0, ignores this variable
host = !hostname -i
host = host[0]
config_path = "/content/config_dirs.txt"

# Define Agent Here
class CustomAgent(base_agent.BaseAgent):
  def step(self, obs):
    super(CustomAgent, self).step(obs)

    # Get the position of the enemy agent
    enemy_position = [obs.observation["enemy_unit"].position_x,
                      obs.observation["enemy_unit"].position_y]
    
    # Print the enemy agents HP with our user id
    print("My ID, Enemy HP: %d %f" % \
          (obs.observation["me_unit"].user_id,
           obs.observation["enemy_unit"].current_hp))

    # Ezreal Q at the enemy agents position
    return actions.FunctionCall(2, [[0], enemy_position])

# Setup the Game
players = []
agents = []
for player in player_list.split(","):
  c, t = player.split(".")
  players.append(lol_env.Agent(champion=c, team=t))
  agents.append(CustomAgent())

# Run the Game
with lol_env.LoLEnv(
  host=host,
  map_name=map,
  players=players,
  agent_interface_format=lol_env.parse_agent_interface_format(
    feature_map=feature_map_size,
    feature_move_range=feature_move_range
  ),
  human_observer=False,
  cooldowns_enabled=False,
  config_path=config_path) as env:

    run_loop.run_loop(agents, env, max_episodes=max_episodes, max_steps=max_steps)

## Download LoL Client, Wine and Winetrick

### Download LoL Client

In [ ]:
!wget --no-check-certificate --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1vkrdzSxTN6FPP7A9R65bIEBNefU7vDJL' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1vkrdzSxTN6FPP7A9R65bIEBNefU7vDJL" -O league-of-legends-420.tar.gz && rm -rf /tmp/cookies.txtInstall Winetricks

### Install Wine

In [ ]:
!sudo dpkg --add-architecture i386
!sudo apt update -y
!sudo apt install wine64 wine32 -y

### Install Winetricks

In [ ]:
!sudo apt-get install winetricks -y
!winetricks d3dx9

### Extract LoL Client

In [ ]:
!tar -xzvf league-of-legends-420.tar.gz

## Test League Client

### Install Virtual X Frame Buffer

In [ ]:
!sudo apt-get install xvfb -y

In [ ]:
import os

s = """
XVFB=/usr/bin/Xvfb
XVFBARGS=":1 -screen 0 320x200x24 -ac +extension GLX +render -noreset"
PIDFILE=/var/run/xvfb.pid
case "$1" in
  start)
    export DISPLAY=:1
    echo -n "Starting virtual X frame buffer: Xvfb"
    start-stop-daemon --start --quiet --pidfile $PIDFILE --make-pidfile --background --exec $XVFB -- $XVFBARGS
    echo "."
    ;;
  stop)
    echo -n "Stopping virtual X frame buffer: Xvfb"
    start-stop-daemon --stop --quiet --pidfile $PIDFILE
    rm $PIDFILE
    echo "."
    ;;
  restart)
    $0 stop
    $0 start
    ;;
  *)
        echo "Usage: /etc/init.d/xvfb {start|stop|restart}"
        exit 1
esac

exit 0
"""

with open("./start_xvfb.sh", "w") as f:
  f.write(s)

game_cfg = """
[General]
GameMouseSpeed=4
UserSetResolution=1
EnableAudio=0
BindSysKeys=0
SnapCameraOnRespawn=1
OSXMouseAcceleration=1
AutoAcquireTarget=1
EnableLightFx=0
WindowMode=1
ShowTurretRangeIndicators=1
PredictMovement=0
WaitForVerticalSync=0
Colors=24
Height=200
Width=320
SystemMouseSpeed=0
CfgVersion=4.20.315

[HUD]
ItemShopPrevY=58
ItemShopPrevX=-218
NameTagDisplay=1
ShowChampionIndicator=0
ShowSummonerNames=1
ScrollSmoothingEnabled=0
MiddleMouseScrollSpeed=0.5000
MapScrollSpeed=0.5000
ShowAttackRadius=1
NumericCooldownFormat=1
SmartCastOnKeyRelease=0
EnableLineMissileVis=1
FlipMiniMap=0
ShowAllChannelChat=1
FlashScreenWhenStunned=0
ShowTimestamps=1
ChatScale=100
FlashScreenWhenDamaged=0
ItemShopResizeHeight=0
ItemShopResizeWidth=260
ItemShopPrevResizeHeight=960
ItemShopPrevResizeWidth=1280
ItemShopItemDisplayMode=1
ItemShopStartPane=1

[Performance]
ShadowsEnabled=0
CharacterInking=0
EnableHUDAnimations=0
PerPixelPointLighting=1
EnableParticleOptimizations=0
BudgetOverdrawAverage=10
BudgetSkinnedVertexCount=200000
BudgetSkinnedDrawCallCount=100
BudgetTextureUsage=150000
BudgetVertexCount=500000
BudgetTriangleCount=300000
BudgetDrawCallCount=1000
EnableGrassSwaying=1
EnableFXAA=0
AdvancedShader=0
FrameCapType=2
EffectsQuality=1
GammaEnabled=0
Full3DModeEnabled=1
EnvironmentQuality=1
CharacterQuality=1
AutoPerformanceSettings=0
ShadowQuality=0
GraphicsSlider=-1

[Volume]
PingsVolume=0.62
AmbienceVolume=0.58
MusicVolume=0.47
MasterVolume=0.41

[LossOfControl]
ShowSlows=0

[FloatingText]
Debug_Enabled=1
Absorbed_Enabled=1
OMW_Enabled=1
EnemyTrueDamageCritical_Enabled=1
EnemyMagicalDamageCritical_Enabled=1
EnemyPhysicalDamageCritical_Enabled=1
Experience_Enabled=1
TrueDamageCritical_Enabled=1
MagicalDamageCritical_Enabled=1
PhysicalDamageCritical_Enabled=1
ManaDamage_Enabled=1
ManaHeal_Enabled=1

[Replay]
EnableHelpTip=0
"""

# TODO: Write game_cfg
with open("/content/League-of-Legends-4-20/Config/game.cfg", "w") as f:
  f.write(game_cfg)

!chmod +x start_xvfb.sh
!./start_xvfb.sh start

### Run Client

In [ ]:
!pip install pyautogui
!pip install Xlib

### Write Config Dirs (now set to GameServer and Client)

In [ ]:
# Only need to write config for GameServer here as we don't use the League Client within Colab environment
!touch config_dirs.txt
!printf "[dirs]\ngameserver = /content/LeagueSandbox-RL-Learning/GameServerConsole/bin/Debug/netcoreapp3.0/\nlolclient = /content/League-of-Legends-4-20/RADS/solutions/lol_game_client_sln/releases/0.0.1.68/deploy/" > config_dirs.txt

In [ ]:
import os
os.environ["DISPLAY"] = ":1"

# NOTE: Uncomment this if you want to test the environment before the
#       experiment
RUN_PYLOL_TEST = False

# Get local IP
host = !hostname -i
host = host[0]

# NOTE: You need to manually stop this cell when you want it to stop recording
#       Automating this process is a nightmare and not worth the effort
# NOTE: It takes about 1 minute to actually load into the game because colab
#       is running on 2 vCPUs which is the same as a single core CPU
# NOTE: The game is rendered at 320x200 because we only have software rendering
#       availabe. Surpringly, league adjusts it's HUD quite well to that resolution
#       so you can still more or less see what's going on.
# NOTE: Sometimes portpicker will pick an unavailable port, just restart this cell
#       if that happens

if RUN_PYLOL_TEST:
  !rm ./pylol_test.mp4
  !ffmpeg -f x11grab -video_size 320x200 -i :1 -r 10 ./pylol_test.mp4 & python3 -m pylol.bin.client --max_steps 250 --host $host --agent random --config_path "/content/config_dirs.txt" --run_client

In [ ]:
if RUN_PYLOL_TEST:
  from IPython.display import HTML
  from base64 import b64encode
  mp4 = open('pylol_test.mp4','rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  HTML("""
  <video width=800 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url)

## Install LoLGym

In [ ]:
!rm -rf lolgym # Delete old version in case we're experimenting with lolgym
!git clone https://github.com/MiscellaneousStuff/lolgym.git
!pip3 install -e lolgym/

# Training

Example task trains an agent to maximize it's distance from another agent by running away from it. Although this is a very simple task, it serves as a sanity test to prove that the environment and the A2C PPO model is working properly.

## Run Experiment

In [ ]:
# NOTE: You need to manually stop this cell when you want it to stop recording
#       Automating this process is a nightmare and not worth the effort
# NOTE: It takes about 1 minute to actually load into the game because colab
#       is running on 2 vCPUs which is the same as a single core CPU
# NOTE: The game is rendered at 320x200 because we only have software rendering
#       availabe. Surpringly, league adjusts it's HUD quite well to that resolution
#       so you can still more or less see what's going on.
# NOTE: Sometimes portpicker will pick an unavailable port, just restart this cell
#       if that happens

# Ensure old experiment data and video is deleted
experiment_output = !ls | grep .txt | grep run_away
experiment_output = experiment_output[0] if experiment_output else ""
!rm experiment.mp4 $experiment_output

# Run Experiment!
epochs = 30 # Number of episodes to run experiment
!ffmpeg -f x11grab -video_size 320x200 -i :1 -r 10 ./experiment.mp4 & python3 ./lolgym/examples/full_game_ppo.py --config_path "/content/config_dirs.txt" --host $host --epochs $epochs

## View Experiment Video

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('experiment.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

## Visualise Data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get raw data from experiment logs
experiment_output = !ls | grep .txt | grep run_away
experiment_output = experiment_output[0]
with open(experiment_output) as f:
  rewards = list(filter(bool, f.read().split("+")))
  data = [float(d) for d in rewards]

# Plot Trendline (PPO can be noisy with low sample size
# so trendline makes graph clearer. Beware outliers)
x = range(len(data))
z = np.polyfit(x, data, 1)
p = np.poly1d(z)
plt.plot(x, p(x), "r--")

# Plot Rewards
plt.xlabel("Episode")
plt.ylabel("Episode Reward (Distance in Units)")
plt.title("Escape Task trained using PPO")
plt.plot(data, label="Mean Episode Reward")